In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os

In [2]:
browser = webdriver.Chrome('C:/Users/curtb/Projects/Mission-To-Mars/chromedriver')

In [3]:
browser.get('https://s3.amazonaws.com/tripdata/index.html')

In [4]:
#get all links
all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [5]:
# extract necessary links
linksJC_17 = [link for link in all_links if 'JC' in link if '2017' in link]
linksJC_18 = [link for link in all_links if 'JC' in link if '2018' in link]
#links_17 = [link for link in all_links if 'JC' not in link if '2017' in link]
#links_18 = [link for link in all_links if 'JC' not in link if '2018' in link]

In [6]:
# function for downloading, unziping and dataframing data from a link
def create_df(link):
    
    url = urllib.request.urlopen(link)
    output = open('temporary.zip', 'wb')    
    output.write(url.read())
    output.close()
    dataframe = pd.read_csv('temporary.zip')
    
    if (len(dataframe.columns) == 15):
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth_Year', 'Gender']
    else:
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Localized Value', 'User Type', 'Birth_Year', 'Gender']
        dataframe = dataframe.drop('Localized Value',1)
        
    print(link)
    os.remove('temporary.zip')
    
    return dataframe

In [7]:
# function for cleaning and preparing df
def prepare_df(df):
    
#     drop n/a
    df = df.dropna(how='any').reset_index(drop=True)
    
#     change data types
    df['Birth_Year'] = df.Birth_Year.astype(int)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Stop Time'] = pd.to_datetime(df['Stop Time'])
    
#     add Age column
    df['Age'] = 2019 - df['Birth_Year']
    
#     exclude ages > 90 years
    df = df[df['Age'] < 90]
    
    return df

In [10]:
# create empty df
JC17 = pd.DataFrame()
JC18 = pd.DataFrame()

In [11]:
# append to new df
for link in linksJC_17:
    temporary_df = create_df(link)
    JC17 = JC17.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/JC-201701-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201702-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201703-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201704-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201705-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201706-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201707-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201708%20citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201709-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201710-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201711-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201712-citibike-tripdata.csv.zip


In [12]:
for link in linksJC_18:
    temporary_df = create_df(link)
    JC18 = JC18.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201803-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201804-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201805-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201806-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201807-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201808-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201809-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201810-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201811-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201812-citibike-tripdata.csv.zip


In [13]:
# clean and prepare df
JC17 = prepare_df(JC17)
JC18 = prepare_df(JC18)

In [14]:
JC17.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender,Age
0,148,2017-01-01 00:21:32,2017-01-01 00:24:01,3276,Marin Light Rail,40.714584,-74.042817,3185,City Hall,40.717733,-74.043845,24575,Subscriber,1983,1,36
1,1283,2017-01-01 00:24:35,2017-01-01 00:45:58,3183,Exchange Place,40.716247,-74.033459,3198,Heights Elevator,40.748716,-74.040443,24723,Subscriber,1978,1,41
2,372,2017-01-01 00:38:19,2017-01-01 00:44:31,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,24620,Subscriber,1989,1,30
3,1513,2017-01-01 00:38:37,2017-01-01 01:03:50,3194,McGinley Square,40.725340,-74.067622,3271,Danforth Light Rail,40.692640,-74.088012,24668,Subscriber,1961,1,58
4,639,2017-01-01 01:47:52,2017-01-01 01:58:31,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,26167,Subscriber,1993,1,26


In [15]:
JC18.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender,Age
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,27
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,50
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1,73
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1,25
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1,28


In [16]:
JC17.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int32
Gender                              int64
Age                                 int32
dtype: object

In [17]:
JC18.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int32
Gender                              int64
Age                                 int32
dtype: object

In [18]:
JCdf = JC17.append(JC18, ignore_index=True, sort=False)

In [19]:
JCdf.count()

Trip Duration (sec)        628885
Start Time                 628885
Stop Time                  628885
Start Station ID           628885
Start Station Name         628885
Start Station Latitude     628885
Start Station Longitude    628885
End Station ID             628885
End Station Name           628885
End Station Latitude       628885
End Station Longitude      628885
Bike ID                    628885
User Type                  628885
Birth_Year                 628885
Gender                     628885
Age                        628885
dtype: int64

In [20]:
JCdf.to_csv('data/JC_citibike_from_url.csv', encoding='utf-8', index=False)